# Model evaluation

In [3]:
%cd models/PGPR

/home/jfallmann/userdata/Bachelor Thesis/bachelor-thesis/src/models/PGPR


In [4]:
import torch
from train_agent import ActorCritic, BatchKGEnvironment

In [7]:
env = BatchKGEnvironment('lfm', 250, max_path_len=3,
                             state_history=1)
device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'
pretrain_sd = torch.load('../../data/lfm/preprocessed/pgpr/tmp/train_agent/policy_model_epoch_50.ckpt')
model = ActorCritic(env.state_dim, env.act_dim, gamma=0.99, hidden_sizes=[512, 256]).to(device)
model_sd = model.state_dict()
model_sd.update(pretrain_sd)
model.load_state_dict(model_sd)

Load embedding: ../../data/lfm/preprocessed/pgpr/tmp/transe_embed.pkl


<All keys matched successfully>

In [12]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

401404

In [14]:
%cd ../CAFE

/home/jfallmann/userdata/Bachelor Thesis/bachelor-thesis/src/models/CAFE


In [39]:
from cafe_utils import *
from data_utils import KGMask
from symbolic_model import EntityEmbeddingModel, SymbolicNetwork

In [40]:
kg = load_kg("lfm")
kg_mask = KGMask(kg)
entity_info, relation_info = {}, {}
for entity in kg.G:
    entity_info[entity] = {"vocab_size": len(kg.G[entity])}
for rel in kg.relation_info:
    relation_info[rel] = {
       "name": rel,
        "entity_head": kg.relation_info[rel][0],
        "entity_tail": kg.relation_info[rel][1],
    }

# pretrain_embeds = utils.load_embed(args.dataset) if train else None
entity_embed_model = EntityEmbeddingModel(entity_info, 100, init_embed=None)
model = SymbolicNetwork(relation_info, entity_embed_model, True,True,device)
model = model.to(device)

pretrain_sd = torch.load("../../data/lfm/preprocessed/cafe/tmp/neural_symbolic_model/symbolic_model_epoch19.ckpt", map_location=lambda storage, loc: storage)
model_sd = model.state_dict()
model_sd.update(pretrain_sd)
model.load_state_dict(model_sd)
model.eval()



SymbolicNetwork(
  (embedding): EntityEmbeddingModel(
    (user): Embedding(9456, 100, padding_idx=9455)
    (product): Embedding(173789, 100, padding_idx=173788)
    (genre): Embedding(750, 100, padding_idx=749)
    (album): Embedding(45979, 100, padding_idx=45978)
    (micro_genre): Embedding(1922, 100, padding_idx=1921)
    (artist): Embedding(14988, 100, padding_idx=14987)
  )
  (listened): DeepRelationModule(
    (fc1): Linear(in_features=200, out_features=256, bias=True)
    (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc2): Linear(in_features=256, out_features=200, bias=True)
    (bn2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc3): Linear(in_features=200, out_features=100, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (rev_listened): DeepRelationModule(
    (fc1): Linear(in_features=200, out_features=256, bias=True)
    (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, af

In [41]:
sum(p.numel() for p in model.parameters() if p.requires_grad)


25927080

# Path Evaluation